In [2]:
from selenium import webdriver
import time
import csv
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import contractions
import string

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\thoma\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
#Scrape 1
browser = webdriver.Chrome()

combined_ratings_reviews_list = []

title_list = []
review_list = []
rate_list = []

url_part_1 = "https://www.amazon.com/Collagen-Peptides-Grass-Fed-Certified-Friendly/product-reviews/B00XQ2XGAA/ref=cm_cr_getr_d_paging_btm_next_3?ie=UTF8&reviewerType=all_reviews"
url_part_2 = "&pageNumber="

for a in range (0,501):
    
    url = url_part_1 + str(a) + url_part_2 + str(a)

    browser.get(url)

    # Give the page 3 sec to load
    time.sleep(2)
    
    html = browser.page_source

    soup = BeautifulSoup(html, "lxml")
    
    containers = soup.find_all('div', {'data-hook':'review'})
    
    for f in range (0, len(containers)):
        f_container = containers[f]
        
        title_text = f_container.find('a', {'data-hook':'review-title'}).find('span').text
        
        review_text = f_container.find('span', class_ = 'a-size-base review-text review-text-content').find('span').text
        
        rating_raw = int(f_container.find('span', class_ ="a-icon-alt").text[0])

        rating_text = ''

        if rating_raw == 5:
            rating_text = 'pos'
        elif rating_raw == 4:
            rating_text = 'pos'
        elif rating_raw == 3:
            rating_text = ' '
        elif rating_raw == 2:
            rating_text = 'neg'
        elif rating_raw == 1:
            rating_text = 'neg'
            
        title_review_combined = str(title_text) + ' ' + str(review_text)
            
        combined_ratings_reviews_list.append([title_review_combined, rating_text])
        title_list.append(title_text)
        review_list.append(review_text)
        rate_list.append(rating_text)
    
browser.close()

In [42]:
len(title_list)

4990

In [44]:
#in case you want to save the scraped data as csv


#DO NOT REUSE CODE BECAUSE IT WILL OVERWRITE CSV 
#with open('Amazon Reviews Dirty.csv', 'w', newline='', encoding='utf-8') as csvfile:
    #wf = csv.writer(csvfile, delimiter=',',
                            #quotechar='"')
    
    #for save in range(0,4990):
        #wf.writerow([title_list[save], review_list[save], rate_list[save].encode("utf-8")])

#how to open Amazon Reviews csv




In [63]:
df = pd.read_csv('Amazon Reviews Dirty.csv', encoding = 'utf-8', names=["Title", "Review", "Rate"])
df

,Title,Review,Rate
0,Fill scooper 80% to get accurate serving size!,I read a few reviews before ordering this prod...,b'pos'
1,35 Day Review,I am a 57 year old woman who has been comforta...,b'pos'
2,I gotta be honest...,So I bought this with the complete intentions ...,b'pos'
3,I really dislike these powder supplements and ...,I received an email from Sports Research askin...,b'pos'
4,IBS is no longer an issue for me,After 15 plus years of battling IBS I felt i w...,b'pos'
5,This product cured my constant hunger and crav...,I try new products monthly. My cupboards are f...,b'pos'
6,THIS is the stuff to get,"I bought this for my joints, with the hope it ...",b'pos'
7,Eat this.,I gave it a full month before leaving a review...,b'pos'
8,REDUCED CELLULITE!!!,It was a girlfriend that let me in on her litt...,b'pos'
9,The Holy Grail,I started using this product every day 45 days...,b'pos'


In [64]:
#delete ratings with 3 (3 is blank in this dataframe)

df.drop('Title',axis=1,inplace=True)

df['Rate'] = df['Rate'].map(lambda x:
                             x.lstrip("b'").rstrip("'"))

df=df[df.Rate != ' ']

df

,Review,Rate
0,I read a few reviews before ordering this prod...,pos
1,I am a 57 year old woman who has been comforta...,pos
2,So I bought this with the complete intentions ...,pos
3,I received an email from Sports Research askin...,pos
4,After 15 plus years of battling IBS I felt i w...,pos
5,I try new products monthly. My cupboards are f...,pos
6,"I bought this for my joints, with the hope it ...",pos
7,I gave it a full month before leaving a review...,pos
8,It was a girlfriend that let me in on her litt...,pos
9,I started using this product every day 45 days...,pos


In [65]:
adf=df.dropna()
adf

,Review,Rate
0,I read a few reviews before ordering this prod...,pos
1,I am a 57 year old woman who has been comforta...,pos
2,So I bought this with the complete intentions ...,pos
3,I received an email from Sports Research askin...,pos
4,After 15 plus years of battling IBS I felt i w...,pos
5,I try new products monthly. My cupboards are f...,pos
6,"I bought this for my joints, with the hope it ...",pos
7,I gave it a full month before leaving a review...,pos
8,It was a girlfriend that let me in on her litt...,pos
9,I started using this product every day 45 days...,pos


In [66]:
Review = []
Rating = []

for row in adf["Review"]:
    #tokenize words
    words = word_tokenize(row)
    #remove punctuations
    clean_words = [word.lower() for word in words if word not in set(string.punctuation)]
    #remove stop words
    english_stops = set(stopwords.words('english'))
    characters_to_remove = ['...', "'s", '', 'le', 'u', '’', 'le', 'ca', 'st', 'nd', 'ft',
                            '”', "", ' ', '  ' 'th', 'e', 'p', 'wo', 'wg', 'd', 'gon', 'na',
                            '“', 'n', 'ê', 'pt', 'ot', "'nt","'re","gi"]
    clean_words = [word for word in clean_words if word not in english_stops]
    clean_words = [word for word in clean_words if word not in set(characters_to_remove)]
    #Lematise words
    wordnet_lemmatizer = WordNetLemmatizer()
    lemma_list = [wordnet_lemmatizer.lemmatize(word) for word in clean_words]
    Review.append(lemma_list)

    for row in adf["Rate"]:
        Rating.append(row)

In [67]:
combined = zip(Review, Rating)

In [68]:
def bag_of_words(words):
    return dict([(word, True) for word in words])

In [69]:
Final_Data = []
for r, v in combined:
    bag_of_words(r)
    Final_Data.append((bag_of_words(r),v))

In [70]:
import random
random.shuffle(Final_Data)
print(len(Final_Data))

4784


In [71]:
train_set, test_set = Final_Data[0:188], Final_Data[188:]

import nltk
import collections
from nltk.metrics.scores import (accuracy, precision, recall, f_measure) 
from nltk import metrics

refsets = collections. defaultdict(set)
testsets = collections.defaultdict(set)

classifier = nltk.NaiveBayesClassifier.train(train_set)
 
for i, (feats, label) in enumerate(test_set):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)

print("Naive Bayes Performance with Unigrams ")    
print("Accuracy:",nltk.classify.accuracy(classifier, test_set))

Naive Bayes Performance with Unigrams 
Accuracy: 0.7056135770234987


In [72]:
nb_classifier = nltk.NaiveBayesClassifier.train(train_set)

nbrefset = collections.defaultdict(set)
nbtestset = collections.defaultdict(set)
 
for i, (feats, label) in enumerate(test_set):
    nbrefset[label].add(i)
    observed = nb_classifier.classify(feats)
    nbtestset[observed].add(i)
    
print("UnigramNB Results")
print('Brand Positive Precision:', precision(refsets['Pos'], testsets['Pos']))
print('Brand Positive Recall:', recall(refsets['Pos'], testsets['Pos']))
print('Brand Positive F-measure:', f_measure(refsets['Pos'], testsets['Pos']))
print('Brand Damaging Precision:', precision(refsets['Neg'], testsets['Neg']))
print('Brand Damaging Recall:', recall(nbtestset['Neg'], nbrefset['Neg']))
print('Brand Damaging F-measure:', f_measure(refsets['Neg'], testsets['Neg']))
print("")

UnigramNB Results
Brand Positive Precision: None
Brand Positive Recall: None
Brand Positive F-measure: None
Brand Damaging Precision: None
Brand Damaging Recall: None
Brand Damaging F-measure: None



In [73]:
classifier.show_most_informative_features(n=50)

Most Informative Features
                     may = True              neg : pos    =     15.2 : 1.0
                    16oz = True              neg : pos    =     11.5 : 1.0
                   color = True              neg : pos    =     11.5 : 1.0
                    find = True              neg : pos    =     11.5 : 1.0
                    away = True              neg : pos    =     11.5 : 1.0
                received = True              neg : pos    =     10.9 : 1.0
               different = True              neg : pos    =      8.9 : 1.0
                    glad = True              neg : pos    =      6.9 : 1.0
                   write = True              neg : pos    =      6.9 : 1.0
                   sound = True              neg : pos    =      6.9 : 1.0
                  course = True              neg : pos    =      6.9 : 1.0
                      oh = True              neg : pos    =      6.9 : 1.0
                 current = True              neg : pos    =      6.9 : 1.0